## Домашнее задание 5



**Задание 2.** Проверить, насколько хорошо работает NER

Данные брать из http://www.labinform.ru/pub/named_entities/

1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.
  
  a. передаём в сетку токен и его соседей.
  
  b. передаём в сетку только токен.

  c. свой вариант.



3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


### Решение:

Загрузим датасет.

In [1]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip

--2022-08-30 19:05:26--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.91MB/s    in 0.9s    

2022-08-30 19:05:27 (1.91 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection

Установим и импортируем библиотеки необходимые для решения поставленной задачи.

In [2]:
!pip install natasha corus
!pip -q install spacy
!python -m spacy download ru_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 1.3 MB/s 
     |████████████████████████████████| 83 kB 2.9 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 55 kB 5.6 MB/s 
     |████████████████████████████████| 41 kB 109 kB/s 
     |████████████████████████████████| 8.2 MB 31.3 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=9f37b54eacc05dfe1c6278a95a6e82859cdebe79bda0aafbe57a47c78034f02c
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=d46cf067e181e3d2b2c26ede540b9f1396ca11c69202aaba0ec978d4d6cbe513
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built interval

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk

nltk.download('averaged_perceptron_tagger_ru')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('tagsets')

from corus import load_ne5
from razdel import tokenize
from sklearn.metrics import classification_report

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [14]:
dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='249',
    text='Д.Медведев уволил 15 высокопоставленных милиционеров\r\n\r\nДмитрий Медведев провел новые увольнения в Министерстве внутренних дел. Как сообщает пресс-служба главы государства, президент подписал 1 апреля указ об освобождении от должностей 15 высокопоставленных милиционеров.\r\n\r\nО том, связаны ли причины отставок с переаттестацией милиционеров в полицейских, не сообщается.\r\n\r\nСреди уволенных:\r\n\r\n- начальник Национального центрального бюро Интерпола при МВД РФ, генерал-майор милиции Тимур Лахонин;\r\n\r\n- заместитель начальника ГУВД по Москве по экономической безопасности, генерал-лейтенант милиции Виктор Васильев;\r\n\r\n- начальник управления по налоговым преступлениям Главного управления внутренних дел по Москве, полковник милиции Юрий Васильев;\r\n\r\n- начальник управления по борьбе с экономическими преступлениями Главного управления внутренних дел по Москве, генерал-майор милиции Александр Данилов;\r\n\r\n- начальник управления внутре

#### **nltk**

Насколько я понял несмотря на то, что библиотекой **nltk** поддерживается определение тэгов для русскоязычных текстов распознавание именнованых сущностей в ее рамках реализовано исключительно для английского языка. Как видим на примере ниже, метод *ne_chunk* не выдал результатов для документа из датасета.

In [15]:
records = load_ne5(dir)
for ix, rec in enumerate(records):
  print(rec.text)
  print('\nИменованные сущности:')
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text), lang='rus')):
    if hasattr(chunk, 'label'):
      print(f'{chunk} - {chunk.label()}')
  break

Д.Медведев уволил 15 высокопоставленных милиционеров

Дмитрий Медведев провел новые увольнения в Министерстве внутренних дел. Как сообщает пресс-служба главы государства, президент подписал 1 апреля указ об освобождении от должностей 15 высокопоставленных милиционеров.

О том, связаны ли причины отставок с переаттестацией милиционеров в полицейских, не сообщается.

Среди уволенных:

- начальник Национального центрального бюро Интерпола при МВД РФ, генерал-майор милиции Тимур Лахонин;

- заместитель начальника ГУВД по Москве по экономической безопасности, генерал-лейтенант милиции Виктор Васильев;

- начальник управления по налоговым преступлениям Главного управления внутренних дел по Москве, полковник милиции Юрий Васильев;

- начальник управления по борьбе с экономическими преступлениями Главного управления внутренних дел по Москве, генерал-майор милиции Александр Данилов;

- начальник управления внутренних дел по Восточному административному округу Главного управления внутренних дел 

#### **spacy**

Проведем необходимую предобработку данных и создадим файлы с BIO разметкой.

In [16]:
docs = []
for rec in records:
    words = []
    labels = []
    idx_ent = -1
    len_ents = len(rec.spans)
    rec_entities = sorted(rec.spans, key=lambda v: v.start)
    ent = None
    is_start = None
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        if len_ents == 0:
            words.append(token.text)
            labels.append(type_ent)
            continue

        if (idx_ent == -1) or (idx_ent + 1 < len_ents and token.start > ent.stop):
            idx_ent += 1
            ent = rec_entities[idx_ent]
            is_start = True

        if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = 'B-' + ent.type if is_start else 'I-' + ent.type
                is_start = False
        words.append(token.text)
        labels.append(type_ent)
    
    docs.append([words, labels])

In [17]:
print(docs[0][0])
print(docs[0][1])

['Врио', 'главы', 'Дагестана', 'намерен', 'бороться', 'с', 'клановостью', 'во', 'власти', 'Временно', 'исполняющий', 'обязанности', 'президента', 'Дагестана', 'Рамазан', 'Абдулатипов', 'намерен', 'бороться', 'с', 'так', 'называемой', '"', 'клановостью', '"', 'во', 'властных', 'структурах', 'республики', '.', 'Он', 'отметил', ',', 'что', 'порой', 'такие', 'кланы', 'во', 'власти', 'превращаются', 'в', 'банды', '.', 'Рамазан', 'Абдулатипов', '.', 'Архив', 'Временно', 'исполняющий', 'обязанности', 'президента', 'Дагестана', 'Рамазан', 'Абдулатипов', 'намерен', 'бороться', 'с', 'так', 'называемой', '"', 'клановостью', '"', 'во', 'властных', 'структурах', 'республики', '.', '"', 'Другое', 'дело', ',', 'когда', 'мне', 'говорят', ',', 'мол', ',', 'надо', 'свою', 'команду', 'создавать', '…', 'И', 'вы', 'знаете', '—', 'вот', 'сначала', 'создают', 'команду', ',', 'а', 'потом', 'эта', 'команда', 'превращается', 'в', 'банду', '.', 'Это', 'и', 'есть', 'клановость', '.', 'Они', 'не', 'родственники', 

In [10]:
training_coeff = 0.75

with open('c5.bio', 'w') as w:
    with open('c5_train.bio', 'w') as w1:
        with open('c5_valid.bio', 'w') as w2:
            for irec, rec in enumerate(docs):
                for line in map(lambda vl: '\t'.join(vl) + '\n', zip(*rec)):
                    w.write(line)
                    if irec < len(docs) * training_coeff:
                        w1.write(line)
                    else:
                        w2.write(line)
                w.write('\n')
                if irec < len(docs) * training_coeff:
                    w1.write('\n')
                else:
                    w2.write('\n')

In [11]:
!python -m spacy init config base_config.cfg -F -p  ner -l ru
!python -m spacy init fill-config base_config.cfg config.cfg
# !python -m spacy convert c5.bio . -t json -c ner
!python -m spacy convert c5_train.bio . -c ner
!python -m spacy convert c5_valid.bio . -c ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: ru
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (750 documents): c5_train.spacy

Обучаем модель.

In [12]:
!python -m spacy train config.cfg --output ./output --paths.train c5_train.spacy --paths.dev c5_valid.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-08-30 19:10:30,795] [INFO] Set up nlp object from config
[2022-08-30 19:10:30,805] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-08-30 19:10:30,811] [INFO] Created vocabulary
[2022-08-30 19:10:30,813] [INFO] Finished initializing nlp object
[2022-08-30 19:10:34,770] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    456.85   79.70   78.73   80.70    0.80
  0     200       1983.08   9789.79   93.89   94.58   93.22    0.94
  0     400       1882.58  

Посмотрим на результаты обученной модели.

In [13]:
!python -m spacy evaluate output/model-last c5_valid.spacy

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -    
NER P   97.83
NER R   98.00
NER F   97.91
SPEED   26893


=============================== NER (per type) ===============================

               P       R       F
T          99.09   99.30   99.20
ORG        80.64   82.48   81.55
PER        93.45   91.47   92.45
GEOPOLIT   90.98   90.52   90.75
MEDIA      82.62   82.40   82.51
LOC        80.89   82.04   81.46



#### **deeppavlov**

Прежде всего установим библиотеку и все необходимые зависимости.

In [60]:
!pip install deeppavlov
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 878 kB 8.5 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 6.7 MB 54.4 MB/s 
     |████████████████████████████████| 1.5 MB 51.6 MB/s 
     |████████████████████████████████| 20.1 MB 70.3 MB/s 
     |████████████████████████████████| 510 kB 51.3 MB/s 
     |████████████████████████████████| 2.9 MB 53.2 MB/s 
     |████████████████████████████████| 654 kB 65.4 MB/s 
     |████████████████████████████████| 57 kB 6.8 MB/s 
     |████████████████████████████████| 26.1 MB 1.4 MB/s 
     |████████████████████████████████| 55 kB 4.7 MB/s 
     |████████████████████████████████| 7.3 MB 48.0 MB/s 
     |████████████████████████████████| 10.4 MB 51.8 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 859 kB 54.5 MB/s 
     |████████████████████████████████

2022-08-30 19:53:09.768 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 1.5 kB/s 
     |████████████████████████████████| 503 kB 66.9 MB/s 
     |████████████████████████████████| 3.8 MB 72.0 MB/s 
     |████████████████████████████████| 50 kB 8.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=dac58f0292db76a569d5b0c7ad9f753e475dbefbedc240e09b008a9174a36b91
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator

Так как модели в библиотеке deeppavlov весьма тяжелые и долго обучаются, будем работать только с небольшой выборкой из нашего датасета.

In [4]:
!pip install tensorflow==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 31 kB/s 
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [2]:
import deeppavlov
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_few_shot_ru, download=True)

2022-08-30 19:59:16.2 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz download because of matching hashes


In [4]:
from deeppavlov.core.commands.utils import parse_config
config_dict = parse_config(configs.ner.ner_few_shot_ru)
print(config_dict['dataset_reader']['data_path'])

~/.deeppavlov/downloads/ner_few_shot_data/


In [8]:
!mkdir /root/.deeppavlov/downloads/ner_few_shot_data

In [18]:
with open('/root/.deeppavlov/downloads/ner_few_shot_data/all.txt', 'w') as w:
    with open('/root/.deeppavlov/downloads/ner_few_shot_data/train.txt', 'w') as w1:
        with open('/root/.deeppavlov/downloads/ner_few_shot_data/valid.txt', 'w') as w2:
          with open('/root/.deeppavlov/downloads/ner_few_shot_data/test.txt', 'w') as w3:
            for irec, rec in enumerate(docs):
                for line in map(lambda vl: '\t'.join(vl) + '\n', zip(*rec)):
                    w.write(line)
                    if irec < 40:
                        w1.write(line)
                    elif irec < 45:
                        w2.write(line)
                    elif irec < 50:
                        w3.write(line)
                w.write('\n')
                if irec < 40:
                    w1.write(line)
                elif irec < 45:
                    w2.write(line)
                elif irec < 50:
                    w3.write(line)

In [19]:
!ls /root/.deeppavlov/downloads/ner_few_shot_data/

all.txt  test.txt  train.txt  valid.txt


In [20]:
!python -m deeppavlov train ner_few_shot_ru

2022-08-30 20:04:46.30 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_few_shot_ru' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/ner/ner_few_shot_ru.json'
2022-08-30 20:04:46.264 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: FitTrainer got additional init parameters ['epochs', 'validation_patience', 'val_every_n_epochs', 'log_every_n_epochs'] that will be ignored:
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2022-08-30 20:04:47.893 INFO in 'deeppavlov.core

In [25]:
from deeppavlov import train_model
ner_model = train_model(configs.ner.ner_few_shot_ru)

2022-08-30 20:13:00.394 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: FitTrainer got additional init parameters ['epochs', 'validation_patience', 'val_every_n_epochs', 'log_every_n_epochs'] that will be ignored:
INFO:deeppavlov.core.trainers.fit_trainer:FitTrainer got additional init parameters ['epochs', 'validation_patience', 'val_every_n_epochs', 'log_every_n_epochs'] that will be ignored:
2022-08-30 20:13:00.402 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_fs/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_fs/tag.dict]
2022-08-30 20:13:00.418 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_fs/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[saving vocabulary to /root/.deeppavlov/models/ner_fs/tag.dict]
2022-08-30 20:16:16.807 INFO in 'deeppavlo

{"valid": {"eval_examples_count": 1, "metrics": {"ner_f1": 85.3767}, "time_spent": "0:00:33"}}


2022-08-30 20:17:19.966 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_fs/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_fs/tag.dict]


{"test": {"eval_examples_count": 1, "metrics": {"ner_f1": 78.9866}, "time_spent": "0:00:29"}}


In [26]:
y_t = [docs[i][1] for i in range(45, 50)]
y_true = [item for sublist in y_t for item in sublist]

In [27]:
np.unique(np.array(y_true))

array(['B-GEOPOLIT', 'B-LOC', 'B-MEDIA', 'B-ORG', 'B-PER', 'I-LOC',
       'I-MEDIA', 'I-ORG', 'I-PER', 'OUT'], dtype='<U10')

In [28]:
y_p = []
for i in range(45, 50):
  y_p.append(ner_model(docs[i][0])[1])

y_pred = [item for sublist in y_p for item in sublist]
y_pred = [item for sublist in y_pred for item in sublist]

In [29]:
np.unique(np.array(y_pred))

array(['B-B-GEOPOLIT', 'B-B-LOC', 'B-B-MEDIA', 'B-B-ORG', 'B-B-PER',
       'B-I-MEDIA', 'B-I-PER', 'B-OUT'], dtype='<U12')

In [31]:
y_true = np.array(y_true)
y_true[np.char.endswith(y_true, 'MEDIA')]='MEDIA'
y_true[np.char.endswith(y_true, 'GEOPOLIT')]='GEOPOLIT'
y_true[np.char.endswith(y_true, 'LOC')]='LOC'
y_true[np.char.endswith(y_true, 'PER')]='PER'
y_true[np.char.endswith(y_true, 'ORG')]='ORG'
y_true[np.char.endswith(y_true, 'OUT')]='OUT'
np.unique(y_true)

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype='<U10')

In [30]:
y_pred = np.array(y_pred)
y_pred[np.char.endswith(y_pred, 'MEDIA')]='MEDIA'
y_pred[np.char.endswith(y_pred, 'GEOPOLIT')]='GEOPOLIT'
y_pred[np.char.endswith(y_pred, 'LOC')]='LOC'
y_pred[np.char.endswith(y_pred, 'PER')]='PER'
y_pred[np.char.endswith(y_pred, 'ORG')]='ORG'
y_pred[np.char.endswith(y_pred, 'OUT')]='OUT'
np.unique(y_pred)

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype='<U12')

In [32]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

    GEOPOLIT       0.87      0.81      0.84        16
         LOC       0.41      0.39      0.40        31
       MEDIA       1.00      0.64      0.78        11
         ORG       0.94      0.20      0.33        86
         OUT       0.93      1.00      0.96      1238
         PER       0.95      0.78      0.85       121

    accuracy                           0.92      1503
   macro avg       0.85      0.63      0.69      1503
weighted avg       0.92      0.92      0.90      1503



Как видим для классов `GEOPOLIT`, `PER`, `MEDIA` результаты получились довольно неплохими даже несмотря на небольшое количество элементов для обучения. Тем не менее на примере `LOC` и `ORG` видно, что модель требуется в более внушительном объеме данных для обучения и получения по-настоящему хороших результатов.

#### **Нейронные сети**

In [1]:
# !pip install tensorflow==2.2.0
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [4]:
dir = 'Collection5/'
records = load_ne5(dir)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [5]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [6]:
df_words.head(3)

,word,tag
0,Д,PER
1,.,PER
2,Медведев,PER


In [7]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [9]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

# без соседних токенов 
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 3),
    output_sequence_length=seq_len)


# с соседними токенами
vectorize_layer_n13 = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=seq_len)

vectorize_layer_n4 = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=4,
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [15]:
embedding_dim = 128

modeln = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Conv1D(embedding_dim, 3),
    Conv1D(embedding_dim, 2),
    GRU(350),
    Dense(200, activation='relu'),
    Dense(6, activation='softmax')
])

In [16]:
modeln.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [17]:
modeln.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 465s 37ms/step - loss: 0.3329 - accuracy: 0.9007 - val_loss: 0.2307 - val_accuracy: 0.9318
Epoch 2/3
12444/12444 [==============================] - 480s 39ms/step - loss: 0.1543 - accuracy: 0.9538 - val_loss: 0.2540 - val_accuracy: 0.9395
Epoch 3/3
12444/12444 [==============================] - 529s 43ms/step - loss: 0.1251 - accuracy: 0.9624 - val_loss: 0.2816 - val_accuracy: 0.9394


In [18]:
vectorize_layer_n13.adapt(text_data)

modeln13 = Sequential([
    vectorize_layer_n13,
    Embedding(vocab_size, embedding_dim),
    Conv1D(embedding_dim, 3),
    Conv1D(embedding_dim, 2),
    GRU(350),
    Dense(200, activation='relu'),
    Dense(6, activation='softmax')
])

modeln13.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

modeln13.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 477s 38ms/step - loss: 0.3215 - accuracy: 0.9055 - val_loss: 0.2315 - val_accuracy: 0.9332
Epoch 2/3
12444/12444 [==============================] - 455s 37ms/step - loss: 0.1491 - accuracy: 0.9559 - val_loss: 0.2307 - val_accuracy: 0.9388
Epoch 3/3
12444/12444 [==============================] - 461s 37ms/step - loss: 0.1230 - accuracy: 0.9625 - val_loss: 0.4162 - val_accuracy: 0.8903


In [19]:
vectorize_layer_n4.adapt(text_data)

modeln4 = Sequential([
    vectorize_layer_n4,
    Embedding(vocab_size, embedding_dim),
    Conv1D(embedding_dim, 3),
    Conv1D(embedding_dim, 2),
    GRU(350),
    Dense(200, activation='relu'),
    Dense(6, activation='softmax')
])

modeln4.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

modeln4.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 456s 37ms/step - loss: 0.3275 - accuracy: 0.9030 - val_loss: 0.2316 - val_accuracy: 0.9328
Epoch 2/3
12444/12444 [==============================] - 464s 37ms/step - loss: 0.1521 - accuracy: 0.9544 - val_loss: 0.2333 - val_accuracy: 0.9375
Epoch 3/3
12444/12444 [==============================] - 461s 37ms/step - loss: 0.1261 - accuracy: 0.9623 - val_loss: 0.2298 - val_accuracy: 0.9390


Как видим метрики accuracy для всех сетей получились довольно близкими, построим на остальные метрики. 

In [20]:
labels_predict_n = modeln.predict(valid_data)
labels_predict_n13 = modeln13.predict(valid_data)
labels_predict_n4 = modeln4.predict(valid_data)

In [22]:
class_preds_n = np.argmax(tf.nn.softmax(labels_predict_n), axis=1)
class_preds_n13 = np.argmax(tf.nn.softmax(labels_predict_n13), axis=1)
class_preds_n4 = np.argmax(tf.nn.softmax(labels_predict_n4), axis=1)

In [25]:
valid_y = encoder.inverse_transform(valid_y)
class_preds_n = encoder.inverse_transform(class_preds_n)
class_preds_n13 = encoder.inverse_transform(class_preds_n13)
class_preds_n4 = encoder.inverse_transform(class_preds_n4)

In [26]:
print(classification_report(valid_y, class_preds_n))

              precision    recall  f1-score   support

    GEOPOLIT       0.89      0.88      0.88      1079
         LOC       0.83      0.80      0.81      1161
       MEDIA       0.93      0.69      0.79       633
         ORG       0.85      0.56      0.67      3397
         OUT       0.94      0.99      0.97     54751
         PER       0.96      0.72      0.83      5347

    accuracy                           0.94     66368
   macro avg       0.90      0.77      0.83     66368
weighted avg       0.94      0.94      0.94     66368



In [27]:
print(classification_report(valid_y, class_preds_n13))

              precision    recall  f1-score   support

    GEOPOLIT       0.89      0.86      0.88      1079
         LOC       0.81      0.80      0.81      1161
       MEDIA       0.94      0.73      0.82       633
         ORG       0.83      0.55      0.67      3397
         OUT       0.97      0.92      0.94     54751
         PER       0.49      0.88      0.63      5347

    accuracy                           0.89     66368
   macro avg       0.82      0.79      0.79     66368
weighted avg       0.92      0.89      0.90     66368



In [28]:
print(classification_report(valid_y, class_preds_n4))

              precision    recall  f1-score   support

    GEOPOLIT       0.90      0.87      0.88      1079
         LOC       0.85      0.78      0.81      1161
       MEDIA       0.92      0.73      0.82       633
         ORG       0.86      0.55      0.67      3397
         OUT       0.94      0.99      0.97     54751
         PER       0.98      0.70      0.82      5347

    accuracy                           0.94     66368
   macro avg       0.91      0.77      0.83     66368
weighted avg       0.94      0.94      0.93     66368



Как видим показатели precision/recall/f1 тоже довольно близки для построенных сетей, однако, на мой взгляд, первая сеть, которая не берет в расчет соседние токены показала лучшие результаты. Довольно близка к ней по качеству сеть, построенная с параметром ngrams=4. Она даже лучше справляется с детектированием класса `MEDIA`.

Тем не менее стоит отметить, что модель построенная с помощью библиотеки __spacy__ показала самые лучшие результаты.